In [9]:
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
#xml_dir = '/data/darknet/python/' + time_str + '_xml/'
#image_dir = '/data/darknet/python/' + time_str + '_image/'
image_dir = '/data/darknet/python/' + time_str + '_images_replaced_ordinary_obj_with_bkg/'
xml_dir = '/data/darknet/python/xml_20190108_images_replaced_ordinary_obj_with_bkg/'
generated_image_dir = '/data/darknet/python/' + time_str + '_images_replaced_bkg/'
os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
background_dir = '/data/darknet/python/background/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
SEED_BKG_DIR = background_dir
GENE_IMG_DIR = generated_image_dir


def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
seed_bkg_names.sort()
seed_xml_names.sort()
for xml_name in seed_xml_names:
    xml_name_part = xml_name[-15:-4]
    print(xml_name_part)
    if not (int(xml_name_part) >= 442000):
        continue
    background_img_data = cv2.imread(SEED_BKG_DIR + seed_bkg_names[5])
    #print(background_img_data.shape)
    
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    print(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', bboxes, background_img_data)

00000442005
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.316444_00000442005.jpg
00000442006
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.331073_00000442006.jpg
00000442019
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.529090_00000442019.jpg
00000442021
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.557605_00000442021.jpg
00000442022
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.572009_00000442022.jpg
00000442024
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.600447_00000442024.jpg
00000442025
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.614628_00000442025.jpg
00000442026
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154123.628914_00000442026.jpg
00000442027
/data/darknet/python/20190108_images_replace

00000442129
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.243744_00000442129.jpg
00000442130
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.258197_00000442130.jpg
00000442131
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.272680_00000442131.jpg
00000442132
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.287222_00000442132.jpg
00000442133
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.301753_00000442133.jpg
00000442134
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.316537_00000442134.jpg
00000442135
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.330851_00000442135.jpg
00000442136
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154125.346147_00000442136.jpg
00000442137
/data/darknet/python/20190108_images_replace

00000442327
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.314172_00000442327.jpg
00000442328
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.329242_00000442328.jpg
00000442329
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.344307_00000442329.jpg
00000442331
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.374244_00000442331.jpg
00000442332
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.388761_00000442332.jpg
00000442334
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.417082_00000442334.jpg
00000442342
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.534586_00000442342.jpg
00000442349
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154128.634999_00000442349.jpg
00000442357
/data/darknet/python/20190108_images_replace

00000442549
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.747343_00000442549.jpg
00000442551
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.776093_00000442551.jpg
00000442553
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.807347_00000442553.jpg
00000442554
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.821832_00000442554.jpg
00000442555
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.836578_00000442555.jpg
00000442556
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.851504_00000442556.jpg
00000442557
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.865944_00000442557.jpg
00000442558
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154131.880349_00000442558.jpg
00000442559
/data/darknet/python/20190108_images_replace

00000442644
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.217440_00000442644.jpg
00000442645
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.233453_00000442645.jpg
00000442647
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.265168_00000442647.jpg
00000442648
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.280766_00000442648.jpg
00000442649
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.296881_00000442649.jpg
00000442650
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.312445_00000442650.jpg
00000442651
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.328198_00000442651.jpg
00000442653
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154133.359468_00000442653.jpg
00000442656
/data/darknet/python/20190108_images_replace

...

/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.265416_00000443138.jpg
00000443143
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.345895_00000443143.jpg
00000443144
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.361942_00000443144.jpg
00000443145
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.378647_00000443145.jpg
00000443146
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.395348_00000443146.jpg
00000443150
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.459160_00000443150.jpg
00000443152
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.492182_00000443152.jpg
00000443157
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154141.572223_00000443157.jpg
00000443161
/data/darknet/python/20190108_images_replaced_ordinary_o

/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.313605_00000443326.jpg
00000443327
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.329495_00000443327.jpg
00000443328
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.345455_00000443328.jpg
00000443329
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.361417_00000443329.jpg
00000443330
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.377562_00000443330.jpg
00000443331
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.393675_00000443331.jpg
00000443332
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.410681_00000443332.jpg
00000443333
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154144.426581_00000443333.jpg
00000443334
/data/darknet/python/20190108_images_replaced_ordinary_o

/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.672674_00000443467.jpg
00000443468
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.689729_00000443468.jpg
00000443469
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.706720_00000443469.jpg
00000443470
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.722913_00000443470.jpg
00000443471
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.738831_00000443471.jpg
00000443472
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.755513_00000443472.jpg
00000443473
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.771721_00000443473.jpg
00000443474
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154146.788203_00000443474.jpg
00000443475
/data/darknet/python/20190108_images_replaced_ordinary_o

/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.583834_00000443575.jpg
00000443576
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.600269_00000443576.jpg
00000443579
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.651076_00000443579.jpg
00000443580
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.667332_00000443580.jpg
00000443584
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.732580_00000443584.jpg
00000443585
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.748838_00000443585.jpg
00000443586
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.765215_00000443586.jpg
00000443587
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154148.781638_00000443587.jpg
00000443588
/data/darknet/python/20190108_images_replaced_ordinary_o

/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154155.393182_00000443900.jpg
00000443924
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154155.837559_00000443924.jpg
00000443931
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154155.953973_00000443931.jpg
00000443932
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154155.969687_00000443932.jpg
00000443933
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154155.986409_00000443933.jpg
00000443934
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.002932_00000443934.jpg
00000443935
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.019030_00000443935.jpg
00000443936
/data/darknet/python/20190108_images_replaced_ordinary_obj_with_bkg/20190108154156.035650_00000443936.jpg
00000443937
/data/darknet/python/20190108_images_replaced_ordinary_o